In [1]:
import os
import time
import warnings
import numpy as np
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import time
import matplotlib.pyplot as plt
import pandas as pd

# hide warkings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy Numpy warnings

Using TensorFlow backend.


In [2]:
# open file and load data
sinwav = 'sinwave.csv'
with open(sinwav,'rb') as sin:
    f = sin.read()
    data = f.decode().split('\n')

data = list(range(1,1001))

In [3]:
# convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq, n_split):
    # extract raw values
    try:
        raw_values = series.values
    except:
        raw_values = np.array(series)
    raw_values = raw_values.reshape(len(raw_values), 1)
    # transform into supervised learning problem X, y
    supervised = series_to_supervised(raw_values, n_lag, n_seq)
    supervised_values = supervised.values
    # split into train and test sets
    train_size = int(len(supervised_values) * n_split)
    print (len(supervised_values))
    test_size = len(supervised_values) - train_size
    train, test = supervised_values[0:train_size,:], supervised_values[train_size:len(supervised_values),:]
    return train, test

# configure
n_lag = 1
n_seq = 3
n_test = 10
n_split = 0.90
# prepare data
series = data
train, test = prepare_data(series, n_test, n_lag, n_seq, n_split)

997


In [4]:
X, y = train[:, 0:n_lag], train[:, n_lag:]

In [8]:
y.shape[1]

3